In [109]:
import nltk
from collections import Counter
import pandas as pd

import csv
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import regexp_tokenize, wordpunct_tokenize,blankline_tokenize
from nltk import PorterStemmer, LancasterStemmer, SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
import re
import string
from collections import Counter
import json
import re as regex
# This will score 0.556+ on the LB, you can further raise this score
# by using PCA and SVD as features.

# From https://github.com/rhiever/datacleaner
from datacleaner import autoclean
from sklearn.model_selection import GridSearchCV, train_test_split
from xgboost import XGBRegressor

from sklearn.metrics import r2_score

import numpy as np



In [110]:
class TwitterData_Initialize():
    data = []
    processed_data = []
    wordlist = []

    featureList = []
    fea_vect=[]
    whitelist = ["n't","not"]    
    
    data_model = None
    data_labels = None
    is_testing = False
    
    def initialize(self, csv_file, is_testing_set=False, from_cached=None):
        if from_cached is not None:
            #self.data_model = pd.read_csv(from_cached)
            self.data_model = pd.read_json(from_cached)

            return

        self.is_testing = is_testing_set

        if not is_testing_set:
            #self.data = pd.read_csv(csv_file, header=0, names=["id", "emotion", "text"])
            self.data = pd.read_json(csv_file)

            #self.data = self.data[self.data["emotion"].isin(["positive", "negative", "neutral"])]
        

        self.processed_data = self.data
        self.wordlist = []
        self.data_model = None
        self.data_labels = None
        
    
    
    
    def do_process(self):
        def stem_and_join(row,stemmer=nltk.PorterStemmer()):
            row["spans"] = list(map(lambda str: stemmer.stem(str.lower()), row["spans"]))
            return row
    
        def tokenize_grams(row):
                #remove numbers
                def remove_numbers(text):
                    return re.sub(r'\d+', '', text)

                #20 Function to remove whitespace
                def remove_whitespace(text):
                    return " ".join(text.split())

                # Function to remove punctuations
                def remove_punctuations(text):
                    words = nltk.word_tokenize(text)
                    punt_removed = [w for w in words if w.lower() not in string.punctuation]
                    return " ".join(punt_removed)
#30

                # Function to remove stop words
                def remove_stopwords(text, lang='english'):
                    stop_words = set(stopwords.words('english'))
                    word_tokens = word_tokenize(text)
                    filtered_sentence = [w for w in word_tokens if not w in stop_words]
                    filtered_sentence = []
                    for w in word_tokens:
                        if w not in stop_words:
                            filtered_sentence.append(w)
                
                    ch=" ".join(filtered_sentence)

                    return ch
#40




                # Function to extract n-grams from text
                def get_ngrams(text, n):

                    n_grams = ngrams(nltk.word_tokenize(text), n)
                    list_grams=[ ' '.join(grams) for grams in n_grams]
                    return list_grams


                # Function to apply lemmatization to a list of words
                def words_lemmatizer(text, encoding="utf8"):
                    wordnet_lemmatizer = WordNetLemmatizer()

                    words = nltk.word_tokenize(text)
                    lemma_words = []
                    wl = WordNetLemmatizer()
                    for word in words:
                        pos = find_pos(word)
                        lemma_words.append(wl.lemmatize(word, pos))
                    return " ".join(lemma_words)

                # Function to find part of speech tag for a word
                def find_pos(word):
                    # Part of Speech constants
                    # ADJ, ADJ_SAT, ADV, NOUN, VERB = 'a', 's', 'r', 'n', 'v'
   
                    pos = nltk.pos_tag(nltk.word_tokenize(word))[0][1]
                    # Adjective tags - 'JJ', 'JJR', 'JJS'
                    if pos.lower()[0] == 'j':
                        return 'a'
                    # Adverb tags - 'RB', 'RBR', 'RBS'
                    elif pos.lower()[0] == 'r':
                        return 'r'
                    # Verb tags - 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'
                    elif pos.lower()[0] == 'v':
                        return 'v'
                    # Noun tags - 'NN', 'NNS', 'NNP', 'NNPS'
                    else:
                        return 'n'
    
            
            
                idx=row["spans"]
                #convert to string
                ch=""
                for w in idx :
                    ch="".join(w);
    
                #words_stemmer(ch, type="PorterStemmer", lang="english", encoding="utf8")
                #Convert to lower case
                txt_num=remove_numbers(ch)
    
                txt_ws=remove_whitespace(txt_num)
    
                txt_pun=remove_punctuations(txt_ws)
        
                txt_final=remove_stopwords(txt_pun)

                texte=words_lemmatizer(txt_final)

                n_grams=get_ngrams(texte, 1)
                row["token_spans"] = n_grams
    
                return row
        self.processed_data = self.processed_data.apply(stem_and_join, axis=1)
        self.processed_data = self.processed_data.apply(tokenize_grams, axis=1)
        
    def build_wordlist(self, min_occurrences=0, max_occurences=500, stopwords=nltk.corpus.stopwords.words("english"),
                       whitelist=None):
        self.wordlist = []
        whitelist = self.whitelist if whitelist is None else whitelist
        

        words = Counter()

        for idx in self.processed_data.index:
            words.update(self.processed_data.loc[idx, "token_spans"])

        
                

        word_df = pd.DataFrame(data={"word": [k for k, v in words.most_common() if min_occurrences < v < max_occurences],
                                     "occurrences": [v for k, v in words.most_common() if min_occurrences < v < max_occurences]},
                               columns=["word", "occurrences"])

        word_df.to_csv("wordlist_copie.csv", index_label="idx")
        self.wordlist = [k for k, v in words.most_common() if min_occurrences < v < max_occurences]
        
        
    def build_data_model(self):
        label_column = []
        
        label_column = ["label"]

        columns = label_column + list(
            map(lambda w: w + "_bow",self.wordlist))
        labels = []
        rows = []
        for idx in self.processed_data.index:
            current_row = []

            if True:
                # add label
                current_label = self.processed_data.loc[idx, "sentiment score"]
                labels.append(current_label)
                current_row.append(current_label)

            # add bag-of-words
            tokens = set(self.processed_data.loc[idx, "token_spans"])
            for _, word in enumerate(self.wordlist):
                current_row.append(1 if word in tokens else 0)

            rows.append(current_row)

        self.data_model = pd.DataFrame(rows, columns=columns)
        self.data_labels = pd.Series(labels)
        return self.data_model, self.data_labels


In [111]:
class TwitterData_Initialize_test(TwitterData_Initialize):

    
    
    def do_process(self):
        def stem_and_join(row,stemmer=nltk.PorterStemmer()):
            row["spans"] = list(map(lambda str: stemmer.stem(str.lower()), row["spans"]))
            return row
    
        def tokenize_grams(row):
                #remove numbers
                def remove_numbers(text):
                    return re.sub(r'\d+', '', text)

                #20 Function to remove whitespace
                def remove_whitespace(text):
                    return " ".join(text.split())

                # Function to remove punctuations
                def remove_punctuations(text):
                    words = nltk.word_tokenize(text)
                    punt_removed = [w for w in words if w.lower() not in string.punctuation]
                    return " ".join(punt_removed)
#30

                # Function to remove stop words
                def remove_stopwords(text, lang='english'):
                    stop_words = set(stopwords.words('english'))
                    word_tokens = word_tokenize(text)
                    filtered_sentence = [w for w in word_tokens if not w in stop_words]
                    filtered_sentence = []
                    for w in word_tokens:
                        if w not in stop_words:
                            filtered_sentence.append(w)
                
                    ch=" ".join(filtered_sentence)

                    return ch
#40




                # Function to extract n-grams from text
                def get_ngrams(text, n):

                    n_grams = ngrams(nltk.word_tokenize(text), n)
                    list_grams=[ ' '.join(grams) for grams in n_grams]
                    return list_grams


                # Function to apply lemmatization to a list of words
                def words_lemmatizer(text, encoding="utf8"):
                    wordnet_lemmatizer = WordNetLemmatizer()

                    words = nltk.word_tokenize(text)
                    lemma_words = []
                    wl = WordNetLemmatizer()
                    for word in words:
                        pos = find_pos(word)
                        lemma_words.append(wl.lemmatize(word, pos))
                    return " ".join(lemma_words)

                # Function to find part of speech tag for a word
                def find_pos(word):
                    # Part of Speech constants
                    # ADJ, ADJ_SAT, ADV, NOUN, VERB = 'a', 's', 'r', 'n', 'v'
   
                    pos = nltk.pos_tag(nltk.word_tokenize(word))[0][1]
                    # Adjective tags - 'JJ', 'JJR', 'JJS'
                    if pos.lower()[0] == 'j':
                        return 'a'
                    # Adverb tags - 'RB', 'RBR', 'RBS'
                    elif pos.lower()[0] == 'r':
                        return 'r'
                    # Verb tags - 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'
                    elif pos.lower()[0] == 'v':
                        return 'v'
                    # Noun tags - 'NN', 'NNS', 'NNP', 'NNPS'
                    else:
                        return 'n'
    
            
            
                idx=row["spans"]
                ch="".join(idx)

    
                #words_stemmer(ch, type="PorterStemmer", lang="english", encoding="utf8")
                #Convert to lower case
                txt_num=remove_numbers(ch)
    
                txt_ws=remove_whitespace(txt_num)
    
                txt_pun=remove_punctuations(txt_ws)
        
                txt_final=remove_stopwords(txt_pun)

                texte=words_lemmatizer(txt_final)

                n_grams=get_ngrams(texte, 1)
                row["token_spans"] = n_grams
    
                return row
        self.processed_data = self.processed_data.apply(stem_and_join, axis=1)
        self.processed_data = self.processed_data.apply(tokenize_grams, axis=1)
        
    def build_wordlist(self, min_occurrences=0, max_occurences=500, stopwords=nltk.corpus.stopwords.words("english"),
                       whitelist=None):
        self.wordlist = []
        whitelist = self.whitelist if whitelist is None else whitelist
        

        words = Counter()

        for idx in self.processed_data.index:
            words.update(self.processed_data.loc[idx, "token_spans"])

        
                

        word_df = pd.DataFrame(data={"word": [k for k, v in words.most_common() if min_occurrences < v < max_occurences],
                                     "occurrences": [v for k, v in words.most_common() if min_occurrences < v < max_occurences]},
                               columns=["word", "occurrences"])

        word_df.to_csv("wordlist_test.csv", index_label="idx")
        self.wordlist = [k for k, v in words.most_common() if min_occurrences < v < max_occurences]
        
        
    def build_data_model(self):
        

        columns = list(
            map(lambda w: w + "_bow",self.wordlist))
        rows = []
        for idx in self.processed_data.index:
            current_row = []

            

            # add bag-of-words
            tokens = set(self.processed_data.loc[idx, "token_spans"])
            for _, word in enumerate(self.wordlist):
                current_row.append(1 if word in tokens else 0)

            rows.append(current_row)

        self.data_model = pd.DataFrame(rows, columns=columns)
        return self.data_model


In [112]:
data = TwitterData_Initialize()
data.initialize("Microblog_Trainingdata.json")

len(data.processed_data)

1700

In [113]:
data_test = TwitterData_Initialize_test()
data_test.initialize("Microblogs_Testdata.json")

data_test.processed_data.head(5)
len(data_test.processed_data)

800

In [114]:
data.do_process()

In [115]:
data.processed_data.head(5)

,cashtag,id,sentiment score,source,spans,token_spans
0,$FB,719659409228451840,0.366,twitter,[watching for bounce tomorrow],"[watch, bounce, tomorrow]"
1,$LUV,719904304207962112,0.638,twitter,[record number of passengers served in 2015],"[record, number, passenger, serve]"
2,$NFLX,5329774,-0.494,stocktwits,[out $nflx -.35],[nflx]
3,$DIA,719891468173844480,0.460,twitter,"[looking for a strong bounc, lunchtime rally com]","[lunchtime, rally, com]"
4,$PLUG,20091246,0.403,stocktwits,[very intrigued with the technology and growth...,"[intrigue, technology, growth, potenti]"


In [116]:
data_test.processed_data = data_test.processed_data.drop(15)
data_test.processed_data.head(5)

,cashtag,id,source,spans
0,$COST,709723193125175300,twitter,consumers keep cautious stance
1,$CTRP,719529094652698600,twitter,Close $CTRP @46.16 from 43.55 entry +6%;$;+6%
2,$INTC,39048670,stocktwits,Every Reason to Be Bullish
3,$PANW,37048093,stocktwits,$PANW need anoth $1 to all time high
4,$JRCC,6207860,stocktwits,Long setup


In [117]:
data_test.do_process()

In [118]:
data.build_wordlist()

In [119]:
data.processed_data.head(5)

,cashtag,id,sentiment score,source,spans,token_spans
0,$FB,719659409228451840,0.366,twitter,[watching for bounce tomorrow],"[watch, bounce, tomorrow]"
1,$LUV,719904304207962112,0.638,twitter,[record number of passengers served in 2015],"[record, number, passenger, serve]"
2,$NFLX,5329774,-0.494,stocktwits,[out $nflx -.35],[nflx]
3,$DIA,719891468173844480,0.460,twitter,"[looking for a strong bounc, lunchtime rally com]","[lunchtime, rally, com]"
4,$PLUG,20091246,0.403,stocktwits,[very intrigued with the technology and growth...,"[intrigue, technology, growth, potenti]"


In [120]:
bow, labels = data.build_data_model()
bow.head(5)
len(bow)

1700

In [121]:
data_test.processed_data.head()

,cashtag,id,source,spans,token_spans
0,$COST,709723193125175300,twitter,"[c, o, n, s, u, m, e, r, s, , k, e, e, p, , ...","[consumer, keep, cautious, stance]"
1,$CTRP,719529094652698600,twitter,"[c, l, o, s, e, , $, c, t, r, p, , @, 4, 6, ...","[close, ctrp, entry]"
2,$INTC,39048670,stocktwits,"[e, v, e, r, y, , r, e, a, s, o, n, , t, o, ...","[every, reason, bullish]"
3,$PANW,37048093,stocktwits,"[$, p, a, n, w, , n, e, e, d, , a, n, o, t, ...","[panw, need, anoth, time, high]"
4,$JRCC,6207860,stocktwits,"[l, o, n, g, , s, e, t, u, p]","[long, setup]"


In [122]:
data_test.build_wordlist()

In [123]:
bow_test= data_test.build_data_model()
bow_test.head(5)
len(bow_test)

799

In [124]:
bow=bow.rename(columns={"label":"y"})
bow.head(5)


,y,stock_bow,short_bow,buy_bow,long_bow,look_bow,high_bow,today_bow,sell_bow,new_bow,...,infect_bow,malwar_bow,inc_bow,brings_bow,upscale_bow,concept_bow,ownitdon'ttradeit_bow,skin_bow,call/put_bow,buyer_bow
0,0.366,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.638,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-0.494,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.460,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.403,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [125]:
bow_test.head(5)


,stock_bow,buy_bow,short_bow,high_bow,long_bow,look_bow,call_bow,'s_bow,go_bow,sell_bow,...,many_bow,holder_bow,pennant/symmetrical_bow,progress_bow,sure_bow,thus_bow,believe_bow,flyin_bow,fundementals_bow,poopoo_bow
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [126]:
# Concat the data frames

# This is for autocleaner to work properly

# Since we have categorical variables we will
# need our encoder to label them correctly
# so we must use our encoder on the full
# dataset to avoid having representation
# errors.
bow_clean = bow.append(bow_test)
bow_clean = autoclean(bow_clean)
train, test = bow_clean[0:len(bow)], bow_clean[len(bow):]

In [127]:
train.head(5)

,'auto_bow,'best_bow,'d_bow,'grossly_bow,'highly_bow,'large_bow,'let_bow,'ll_bow,'m_bow,'outperform_bow,...,yoku_bow,zero_bow,zigzag_bow,zmh_bow,znga_bow,zombie_bow,zon_bow,zone_bow,zuck_bow,zuckerberg_bow
0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [128]:
test.head(5)

,'auto_bow,'best_bow,'d_bow,'grossly_bow,'highly_bow,'large_bow,'let_bow,'ll_bow,'m_bow,'outperform_bow,...,yoku_bow,zero_bow,zigzag_bow,zmh_bow,znga_bow,zombie_bow,zon_bow,zone_bow,zuck_bow,zuckerberg_bow
0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [129]:
# Organize our data for training
X = train.drop(["y"], axis=1)
Y = train["y"]
x_test = test.drop(["y"], axis=1)
X, X_Val, Y, Y_Val = train_test_split(X, Y)
print(type(Y_Val))

<class 'pandas.core.series.Series'>


In [132]:
print("X=",len(X),"Y=", len(X_Val),"X_val=", len(Y),"Y_val=", len(Y_Val))

X= 1275 Y= 425 X_val= 1275 Y_val= 425


In [133]:
Y_Val.head(5)

1251   -0.528
1018    0.311
481     0.415
1465    0.579
814    -0.329
Name: y, dtype: float64

In [134]:
# A parameter grid for XGBoost
params = {'min_child_weight':[4,5], 'gamma':[i/10.0 for i in range(3,6)],  'subsample':[i/10.0 for i in range(6,11)],
'colsample_bytree':[i/10.0 for i in range(6,11)], 'max_depth': [2,3,4]}


In [135]:
# Initialize XGB and GridSearch
xgb = XGBRegressor(nthread=-1) 

grid = GridSearchCV(xgb, params)
grid.fit(X, Y)

GridSearchCV(cv=None, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=-1, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'min_child_weight': [4, 5], 'gamma': [0.3, 0.4, 0.5], 'subsample': [0.6, 0.7, 0.8, 0.9, 1.0], 'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0], 'max_depth': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [136]:
# Print the r2 score
print(r2_score(Y_Val, grid.best_estimator_.predict(X_Val))) 

0.34274347426351115


In [138]:
# Save the file
y_test = grid.best_estimator_.predict(x_test)
results_df = pd.DataFrame(data={'y':y_test}) 
#ids = test_df["ID"]
ids=bow_test.index
joined = pd.DataFrame(ids).join(results_df)
joined.to_csv("mercedes.csv", index=False)

In [139]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(Y_Val, y_test)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

ValueError: Found input variables with inconsistent numbers of samples: [425, 799]

In [142]:
#load file data into dictionnary 
with open('Microblog_Trialdata.json') as json_data:
    print(type(json_data))
    data_dict = json.load(json_data)

<class '_io.TextIOWrapper'>


In [143]:
len(data_dict)

11